In [1]:
# imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import xlwt
from xlwt import Workbook

In [22]:
#country of interest
COUNTRY = "Nigeria"

# List of municipalities (sample)
MUNICIPALITIES = ["Uvaha", "Burimari", "Kofa", "Gajigana", "Dalori", "Ngamgam", "Kaula", "Shuwa", "Kaya", "Duhu", "Kuda", "Zabarmari"]

# create excel workbook and sheet:
wb = Workbook()
sheet1 = wb.add_sheet('Municipality and State')
sheet1.write(0, 0, "Municipality")
sheet1.write(0, 1, "State(s)")
sheet1.write(0, 2, "Multiple States?")


In [18]:
#start webdriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


In [23]:
line = 1

#loop through all municipalities and enter the states they are in into the spreadsheet
for municipality in MUNICIPALITIES:

    driver.get('https://www.google.com/maps')

    query = municipality + " " + COUNTRY

    # search in searchbar
    search = driver.find_element(By.ID, 'searchboxinput')
    search.send_keys(query)
    search.send_keys(Keys.RETURN)
    #wait for results
    driver.implicitly_wait(3)

    # this class will appear when only one location is returned
    info = driver.find_elements(By.CLASS_NAME, "bwoZTb")


    states = set()

    ### When only one result is returned:
    if len(info) != 0:

        # input state data if it is in the specified country
        if len(info) > 2:
            country = info[2].text
            if country == COUNTRY:
                state = info[1].text
                states.add(state)

    ### When there are multiple results
    else:
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')

        # class "PbZDve" is the element at the bottom of the results that says "You've reached the end of the list"
        end = driver.find_elements(By.CLASS_NAME, "PbZDve")

        while len(end) == 0: #scroll until end is loaded
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight",scroll)
            driver.implicitly_wait(2)
            end = driver.find_elements(By.CLASS_NAME, "PbZDve")

        #find all search results and add the states to the list for that particulary municipality
        results = driver.find_elements(By.CLASS_NAME, 'W4Efsd')

        states = set()
        for i in range(len(results)):
            if results[i].text == COUNTRY:
                states.add(results[i-1].text)



#     print(municipality, states)


    # write results to excel spreadsheet
    sheet1.write(line, 0, municipality)
    if len(states) == 0:
        sheet1.write(line, 1, "None")
    
    if len(states) == 1:
        for state in states:
            sheet1.write(line, 1, state)
            
    if len(states) > 1:
        state_list = ""
        for state in states:
            state_list += state + ", "
        sheet1.write(line, 1, state_list)
        sheet1.write(line, 2, "Yes")


    line += 1

Uvaha {'Borno'}
Burimari {'Borno'}
Kofa {'Kano'}
Gajigana {'Borno'}
Dalori {'Borno'}
Ngamgam {'Borno'}
Kaula {'Borno'}
Shuwa {'Adamawa'}
Kaya {'Kaduna'}
Duhu {'Adamawa'}
Kuda {'Ogun State', 'Nasarawa', 'Niger', 'Borno', 'Adamawa', 'Zamfara', 'Jigawa'}
Zabarmari set()


In [24]:
wb.save('municipalities_to_states_nigeria.xls')
